In [1]:
import pandas as pd
import numpy as np
import config
from sqlalchemy import text
print(config.version)
print(config.connection_string)
pd.set_option('display.max_columns', None)

3.2.0.Dev
postgresql://postgres:sa@localhost/icip


In [2]:
specialite=10
year=21
dept=6
source="PS"
min=30
accessibilite_exp=-0.12 #0.08 pour 45
print(year, specialite, source)

21 10 PS


In [3]:
sources: dict[str, str] = {
    "PS": f"""
select ps.id, ps.nom, ps.prenom, an.dept_id, an.id adresse_norm_id, i.id iris, an.lon, an.lat from ps
join tarif t on t.ps_id = ps.id
join tarif_date_source tds on tds.tarif_id=t.id 
join cabinet c on t.cabinet_id=c.id
join adresse_raw ar on c.adresse_raw_id=ar.id
join adresse_norm an on ar.adresse_norm_id=an.id
join specialite_profession sp on sp.profession_id=t.profession_id
join iris.iris i on i.code=an.iris
where sp.specialite_id={specialite}
and tds.date_source_id >= {year}00 and  tds.date_source_id < {year+1}00
and ar.dept_id={dept}
group by ps.id, c.id, an.id, i.id
""",
    "PA": f"""
select pa.id, pa.nom, pa.prenom, an.dept_id, an.id adresse_norm_id, i.id iris, an.lon, an.lat from personne_activite pa
join personne_activite_diplome pad on pad.personne_activite_id=pa.id
join diplome d on pad.diplome_id=d.id
join profession_diplome pd on pd.diplome_id=d.id
join profession p on pd.profession_id=p.id
join specialite_profession sp on sp.profession_id=p.id
join pa_adresse_norm_date_source pands ON pands.personne_activite_id = pa.id
join adresse_norm an on pands.adresse_norm_id=an.id
join iris.iris i on i.code=an.iris
where sp.specialite_id={specialite}
and pands.date_source_id/100={year}
and an.dept_id={dept}
group by pa.id, an.id, i.id
""",
    "RPPS": f"""
select personne.id, personne.nom, personne.prenom, an.dept_id, an.id adresse_norm_id, i.id iris, an.lon, an.lat from personne
join diplome_obtenu dio on dio.personne_id = personne.id
join diplome d on d.id=dio.diplome_id
join profession_diplome pd on pd.diplome_id=d.id
join profession p on pd.profession_id=p.id
join specialite_profession sp on sp.profession_id=p.id
join coord c on c.personne_id=personne.id
join adresse_norm an on c.adresse_norm_id=an.id
join iris.iris i on i.code=an.iris
where sp.specialite_id={specialite}
and personne.date_maj > '20{year}-01-01' and date_effet < '20{year}-01-01'
and an.dept_id={dept}
group by personne.id, an.id, i.id
"""}

sql = sources[source]
print(sql)


select ps.id, ps.nom, ps.prenom, an.dept_id, an.id adresse_norm_id, i.id iris, an.lon, an.lat from ps
join tarif t on t.ps_id = ps.id
join tarif_date_source tds on tds.tarif_id=t.id 
join cabinet c on t.cabinet_id=c.id
join adresse_raw ar on c.adresse_raw_id=ar.id
join adresse_norm an on ar.adresse_norm_id=an.id
join specialite_profession sp on sp.profession_id=t.profession_id
join iris.iris i on i.code=an.iris
where sp.specialite_id=10
and tds.date_source_id >= 2100 and  tds.date_source_id < 2200
and ar.dept_id=6
group by ps.id, c.id, an.id, i.id



In [4]:
df = pd.read_sql(sql, config.connection_string)
df

,id,nom,prenom,dept_id,adresse_norm_id,iris,lon,lat
0,16126,LEONARDI,DOMINIQUE,6,1415,60881903,7.255881,43.709653
1,16130,BENSUSSAN,JEAN,6,92990,60882801,7.227639,43.676026
2,16131,LARA,JEANNINE,6,92963,60880202,7.282865,43.695573
3,16132,GUEGAN,JEAN CLAUDE,6,1425,60270102,7.145627,43.657352
4,16148,CROQUET,PATRICE,6,1433,60690109,6.922279,43.651604
...,...,...,...,...,...,...,...,...
1280,155689,SAROUX,DELPHINE,6,2305,60882401,7.258125,43.695605
1281,155690,PERADOTTO,CYRIL,6,2866,60290121,6.988390,43.550361
1282,155692,DONADIO,MANON,6,77494,61490102,7.313438,43.743446
1283,155827,BOUCHET BADEL,MARIE,6,2328,60690102,6.914032,43.646318


In [5]:
nb_ps = df["id"].nunique()
unique_ps = df.groupby(["id", "lon", "lat"])
print(f"Nb PS {nb_ps}")
print(f"Nb unique PS {len(unique_ps)}")

Nb PS 1203
Nb unique PS 1273


In [6]:
df["key"] = df["id"].astype(str) + "_" + df["lat"].astype(str) + "_" + df["lon"].astype(str)
df["nb_cabinet"] = df.groupby("id")["key"].transform("nunique")
df["nb_cabinet"].value_counts(normalize=True)

nb_cabinet
1    0.893385
2    0.094163
3    0.009339
4    0.003113
Name: proportion, dtype: float64

In [7]:
df["weight"] = 1 / df["nb_cabinet"]
df["nb"] = df.groupby("iris")["weight"].transform("sum")
df.head(10)

,id,nom,prenom,dept_id,adresse_norm_id,iris,lon,lat,key,nb_cabinet,weight,nb
0,16126,LEONARDI,DOMINIQUE,6,1415,60881903,7.255881,43.709653,16126_43.709653_7.255881,1,1.0,5.000000
1,16130,BENSUSSAN,JEAN,6,92990,60882801,7.227639,43.676026,16130_43.676026_7.227639,1,1.0,8.000000
2,16131,LARA,JEANNINE,6,92963,60880202,7.282865,43.695573,16131_43.695573_7.282865,1,1.0,3.000000
3,16132,GUEGAN,JEAN CLAUDE,6,1425,60270102,7.145627,43.657352,16132_43.657352_7.145627,1,1.0,5.000000
4,16148,CROQUET,PATRICE,6,1433,60690109,6.922279,43.651604,16148_43.651604_6.922279,1,1.0,10.500000
5,16151,DESGATS,MARC,6,1436,60300107,7.005844,43.579045,16151_43.579045_7.005844,1,1.0,5.000000
6,16153,DUBOIS,MICHEL,6,1438,61360000,7.449651,43.877046,16153_43.877046_7.449651,1,1.0,4.000000
7,16159,GIDEL,PIERRE JEAN,6,1466,60882601,7.242220,43.689322,16159_43.689322_7.24222,1,1.0,11.666667
8,16160,MASCHINO,JEAN MARC,6,1467,60290102,7.037066,43.546353,16160_43.546353_7.037066,1,1.0,3.000000
9,16161,SARDOU,YVES,6,1468,60830205,7.492397,43.791639,16161_43.791639_7.492397,1,1.0,1.500000


In [8]:
df = df.sort_values(by='iris')
df = df.drop_duplicates(subset=['iris', 'nb'])
df


,id,nom,prenom,dept_id,adresse_norm_id,iris,lon,lat,key,nb_cabinet,weight,nb
123,16425,DAEMERS,YVES,6,92989,60040102,7.072160,43.596934,16425_43.596934_7.07216,1,1.0,2.000000
904,113924,DIAMANT,GUY,6,77467,60040103,7.137517,43.567521,113924_43.567521_7.137517,1,1.0,1.000000
109,16404,MANDRILE,DIDIER,6,1651,60040104,7.092388,43.604533,16404_43.604533_7.092388,1,1.0,1.000000
31,16222,NICASTRO,SERGE,6,92975,60040105,7.117538,43.588105,16222_43.588105_7.117538,1,1.0,2.000000
1179,144024,FRICKEY,SARAH,6,2734,60040106,7.112369,43.606074,144024_43.606074_7.112369,1,1.0,8.666667
...,...,...,...,...,...,...,...,...,...,...,...,...
607,17388,ROUSSEAU,FRANCOISE,6,2635,61610103,7.123625,43.658047,17388_43.658047_7.123625,1,1.0,1.000000
1069,132333,AGOSTINI,PIERRE,6,81986,61610104,7.141591,43.641591,132333_43.641591_7.141591,2,0.5,3.000000
742,99028,BENASSAYAG,JEAN JACQUES,6,68080,61610105,7.140329,43.639824,99028_43.639824_7.140329,1,1.0,5.000000
301,16814,DI VINCENZO,DOMINIQUE,6,273951,61620000,7.612673,44.063665,16814_44.063665_7.612673,2,0.5,0.500000


In [9]:
def get_iris_matrix():
    sql = f"""
(select iris.id "iris1", iris.id "iris2", 0 "km", 0 "time", 0 "time_hp" from iris.iris
where id / 10000000 = {dept}
union
select iris_id_from as "iris1", iris_id_to "iris2", route_km "km", route_min "time", route_hp_min "time_hp" from iris.iris_matrix
where (iris_id_from/10000000={dept} or iris_id_to/10000000={dept})
and route_min <= {min}) order by "iris1", "iris2"
"""
    print(sql)
    return pd.read_sql(sql, config.connection_string)

# iris_matrix = get_iris_matrix()
iris_matrix = pd.read_csv("od_06_sql_rename.csv")
iris_matrix["iris"] = iris_matrix["iris2"].astype("int64")
iris_matrix
    

,iris1,iris2,km,time,iris
0,40080000,60530000,17,22,60530000
1,40080000,60990000,21,24,60990000
2,40390000,61540000,24,30,61540000
3,40420000,60530000,11,19,60530000
4,40420000,60710000,22,30,60710000
...,...,...,...,...,...
117764,831380000,60850103,34,29,60850103
117765,831380000,60950101,18,19,60950101
117766,831380000,60950102,19,22,60950102
117767,831380000,60950103,18,20,60950103


In [10]:
def get_pop_iris():
    sql = f"""
select i.id iris, pi.iris iris_string, c.code code_commune, i.type type_iris, pi.p20_pop0002, pi.p20_pop0305, pi.p20_pop0610, pi.p20_pop1117, pi.p20_pop1824, pi.p20_pop2539, pi.p20_pop4054, pi.p20_pop5564, pi.p20_pop6579, pi.p20_pop80p
from iris.pop_iris pi
join iris.iris i on  pi.iris=i.code
join iris.commune c on i.commune_id=c.id
where com like '{dept:02}%'"""
    print(sql)
    return pd.read_sql(text(sql), config.connection_string)

pop_iris = get_pop_iris()
pop_iris


select i.id iris, pi.iris iris_string, c.code code_commune, i.type type_iris, pi.p20_pop0002, pi.p20_pop0305, pi.p20_pop0610, pi.p20_pop1117, pi.p20_pop1824, pi.p20_pop2539, pi.p20_pop4054, pi.p20_pop5564, pi.p20_pop6579, pi.p20_pop80p
from iris.pop_iris pi
join iris.iris i on  pi.iris=i.code
join iris.commune c on i.commune_id=c.id
where com like '06%'


,iris,iris_string,code_commune,type_iris,p20_pop0002,p20_pop0305,p20_pop0610,p20_pop1117,p20_pop1824,p20_pop2539,p20_pop4054,p20_pop5564,p20_pop6579,p20_pop80p
0,60120105,060120105,06012,A,87.418550,60.438694,193.693660,273.95730,223.035700,676.78790,830.08154,389.410980,363.85060,233.009220
1,60500000,060500000,06050,C,14.457494,14.409695,41.336414,48.39683,17.661858,72.61527,146.12288,82.679535,64.89807,26.421946
2,60720000,060720000,06072,C,0.000000,7.000000,5.000000,5.00000,4.000000,22.00000,28.00000,28.000000,33.00000,15.000000
3,61050101,061050101,06105,A,90.654580,136.365310,244.816740,402.70312,211.586750,430.94614,944.59796,513.959000,591.59607,209.481000
4,60150000,060150000,06015,C,36.382305,39.335630,53.229595,83.62182,64.078064,198.46510,268.38065,226.838270,194.07529,71.593270
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,60883403,060883403,06088,A,71.638420,55.635365,104.755210,138.66922,184.708200,294.65110,287.00873,264.283000,267.70267,213.472820
496,60883501,060883501,06088,A,63.991177,86.144790,123.785240,215.96701,182.714100,349.32108,586.11273,378.182900,378.49860,152.377090
497,60883601,060883601,06088,A,100.090430,73.343470,134.026950,215.99605,173.736830,455.11038,490.30264,414.152740,495.68090,194.919170
498,60883602,060883602,06088,A,64.699010,79.082280,140.378510,183.99406,110.133000,398.01132,462.45010,307.656770,423.24580,82.143060


In [11]:
apl = iris_matrix.merge(pop_iris, on="iris", how="left", suffixes=('', ''))
apl

,iris1,iris2,km,time,iris,iris_string,code_commune,type_iris,p20_pop0002,p20_pop0305,p20_pop0610,p20_pop1117,p20_pop1824,p20_pop2539,p20_pop4054,p20_pop5564,p20_pop6579,p20_pop80p
0,40080000,60530000,17,22,60530000,060530000,06053,C,3.000000,3.000000,3.000000,15.00000,3.000000,19.00000,22.00000,20.000000,39.000000,14.00000
1,40080000,60990000,21,24,60990000,060990000,06099,C,45.961033,60.562077,118.186980,174.26068,100.920586,288.92783,375.23218,213.348510,308.132720,140.46739
2,40390000,61540000,24,30,61540000,061540000,06154,C,14.443875,11.072821,23.354239,41.09353,36.612030,60.12047,88.63108,67.417816,107.328766,22.92537
3,40420000,60530000,11,19,60530000,060530000,06053,C,3.000000,3.000000,3.000000,15.00000,3.000000,19.00000,22.00000,20.000000,39.000000,14.00000
4,40420000,60710000,22,30,60710000,060710000,06071,C,13.000000,11.000000,30.000000,25.00000,21.000000,92.00000,105.00000,99.000000,131.000000,58.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117764,831380000,60850103,34,29,60850103,060850103,06085,A,71.068900,71.788710,146.845930,194.39722,148.091400,351.81476,491.94345,358.296630,519.871950,236.63065
117765,831380000,60950101,18,19,60950101,060950101,06095,A,78.000000,88.000000,211.000000,256.00000,155.000000,476.00000,682.00000,415.000000,583.000000,261.00000
117766,831380000,60950102,19,22,60950102,060950102,06095,A,68.000000,78.000000,156.000000,226.00000,145.000000,400.00000,527.00000,351.000000,423.000000,175.00000
117767,831380000,60950103,18,20,60950103,060950103,06095,A,47.000000,69.000000,126.000000,225.00000,103.000000,259.00000,536.00000,385.000000,523.000000,184.00000


In [12]:
accessibilite_fn = lambda x: np.exp(accessibilite_exp * x)

apl["time"] = apl["time"].replace(0, 1)
apl["accessibilite_weight"] = accessibilite_fn(apl["time"])
apl.head(10)

,iris1,iris2,km,time,iris,iris_string,code_commune,type_iris,p20_pop0002,p20_pop0305,p20_pop0610,p20_pop1117,p20_pop1824,p20_pop2539,p20_pop4054,p20_pop5564,p20_pop6579,p20_pop80p,accessibilite_weight
0,40080000,60530000,17,22,60530000,060530000,06053,C,3.000000,3.000000,3.000000,15.000000,3.000000,19.000000,22.000000,20.000000,39.000000,14.000000,0.071361
1,40080000,60990000,21,24,60990000,060990000,06099,C,45.961033,60.562077,118.186980,174.260680,100.920586,288.927830,375.232180,213.348510,308.132720,140.467390,0.056135
2,40390000,61540000,24,30,61540000,061540000,06154,C,14.443875,11.072821,23.354239,41.093530,36.612030,60.120470,88.631080,67.417816,107.328766,22.925370,0.027324
3,40420000,60530000,11,19,60530000,060530000,06053,C,3.000000,3.000000,3.000000,15.000000,3.000000,19.000000,22.000000,20.000000,39.000000,14.000000,0.102284
4,40420000,60710000,22,30,60710000,060710000,06071,C,13.000000,11.000000,30.000000,25.000000,21.000000,92.000000,105.000000,99.000000,131.000000,58.000000,0.027324
5,40420000,60990000,17,24,60990000,060990000,06099,C,45.961033,60.562077,118.186980,174.260680,100.920586,288.927830,375.232180,213.348510,308.132720,140.467390,0.056135
6,40420000,61240000,15,30,61240000,061240000,06124,C,0.941176,0.000000,1.882353,0.941176,0.941176,2.823529,10.352942,6.588235,19.764706,3.764706,0.027324
7,40430000,60020000,15,20,60020000,060020000,06002,C,0.962963,0.000000,0.962963,0.000000,4.814815,2.888889,12.518518,12.518518,14.444445,2.888889,0.090718
8,40430000,60240000,8,11,60240000,060240000,06024,C,4.785714,5.982143,4.785714,17.946428,3.589286,21.535715,39.482143,22.732143,61.017857,19.142857,0.267135
9,40430000,60450000,18,23,60450000,060450000,06045,C,0.000000,0.000000,2.730769,6.371795,1.820513,2.730769,17.294872,16.384615,19.115385,4.551282,0.063292


In [13]:
apl[apl["iris1"]==61290000]

,iris1,iris2,km,time,iris,iris_string,code_commune,type_iris,p20_pop0002,p20_pop0305,p20_pop0610,p20_pop1117,p20_pop1824,p20_pop2539,p20_pop4054,p20_pop5564,p20_pop6579,p20_pop80p,accessibilite_weight
105136,61290000,60730000,14,17,60730000,060730000,06073,C,15.983300,14.973627,42.952587,55.938515,20.985765,96.990200,145.970350,117.074715,107.095375,35.035560,0.130029
105137,61290000,60800000,10,18,60800000,060800000,06080,C,4.952830,1.981132,3.962264,3.962264,2.971698,10.896227,18.820755,16.839622,35.660378,4.952830,0.115325
105138,61290000,60820000,23,30,60820000,060820000,06082,C,1.019608,1.019608,6.117647,5.098039,4.078432,16.313726,27.529411,14.274509,23.450981,5.098039,0.027324
105139,61290000,61020000,13,21,61020000,061020000,06102,C,4.363637,1.454546,3.636364,8.727273,3.636364,10.909091,18.181818,26.909090,24.727272,9.454545,0.080460
105140,61290000,61100000,12,19,61100000,061100000,06110,C,3.000000,3.000000,3.000000,7.000000,5.000000,12.000000,25.000000,36.000000,17.000000,8.000000,0.102284
105141,61290000,61110000,10,17,61110000,061110000,06111,C,0.000000,0.000000,1.949580,3.899160,2.924370,15.596639,23.394958,27.294117,26.319328,14.621849,0.130029
105142,61290000,61290000,0,1,61290000,061290000,06129,C,12.756727,9.751091,21.423054,20.561964,12.997735,51.546980,55.843346,32.213420,72.667990,17.237692,0.886920
105143,61290000,61460000,18,23,61460000,061460000,06146,C,5.000000,3.000000,11.000000,16.000000,7.000000,23.000000,35.000000,28.000000,13.000000,8.000000,0.063292
105144,61290000,61530000,14,21,61530000,061530000,06153,C,18.000000,23.000000,44.000000,67.000000,41.000000,120.000000,165.000000,152.000000,156.000000,48.000000,0.080460


In [14]:
cols = [col for col in apl.columns if "pop" in col]
cols

['p20_pop0002',
 'p20_pop0305',
 'p20_pop0610',
 'p20_pop1117',
 'p20_pop1824',
 'p20_pop2539',
 'p20_pop4054',
 'p20_pop5564',
 'p20_pop6579',
 'p20_pop80p']

In [15]:
sql = f"""
select o.* from apl.overrepresentation o
where o.year={year} and o.psp_spe_snds=1
"""
over = pd.read_sql(sql, config.connection_string)
# todo ajouter la colonne psp_spe_snds dans specialite
over

,year,psp_spe_snds,p0002,p0305,p0610,p1117,p1824,p2539,p4054,p5564,p6579,p80p
0,21,1,1.143012,0.52797,0.398005,0.421737,0.712587,0.925556,0.879935,1.076956,1.525866,2.40061


In [16]:
weights = over.values[0,2:]
weights

array([1.14301231, 0.52796983, 0.39800467, 0.42173695, 0.71258726,
       0.92555555, 0.87993485, 1.07695618, 1.52586644, 2.40061021])

In [17]:
apl["pop_gp"] = sum(w * apl[c] for w, c in zip(weights, cols))
apl.sort_values(by='iris2')
apl.head(5)   

,iris1,iris2,km,time,iris,iris_string,code_commune,type_iris,p20_pop0002,p20_pop0305,p20_pop0610,p20_pop1117,p20_pop1824,p20_pop2539,p20_pop4054,p20_pop5564,p20_pop6579,p20_pop80p,accessibilite_weight,pop_gp
0,40080000,60530000,17,22,60530000,060530000,06053,C,3.000000,3.000000,3.000000,15.00000,3.000000,19.00000,22.00000,20.000000,39.000000,14.00000,0.071361,166.271356
1,40080000,60990000,21,24,60990000,060990000,06099,C,45.961033,60.562077,118.186980,174.26068,100.920586,288.92783,375.23218,213.348510,308.132720,140.46739,0.056135,1911.697288
2,40390000,61540000,24,30,61540000,061540000,06154,C,14.443875,11.072821,23.354239,41.09353,36.612030,60.12047,88.63108,67.417816,107.328766,22.92537,0.027324,500.115348
3,40420000,60530000,11,19,60530000,060530000,06053,C,3.000000,3.000000,3.000000,15.00000,3.000000,19.00000,22.00000,20.000000,39.000000,14.00000,0.102284,166.271356
4,40420000,60710000,22,30,60710000,060710000,06071,C,13.000000,11.000000,30.000000,25.00000,21.000000,92.00000,105.00000,99.000000,131.000000,58.00000,0.027324,681.401551


In [18]:
apl[apl["iris1"]==61290000]

,iris1,iris2,km,time,iris,iris_string,code_commune,type_iris,p20_pop0002,p20_pop0305,p20_pop0610,p20_pop1117,p20_pop1824,p20_pop2539,p20_pop4054,p20_pop5564,p20_pop6579,p20_pop80p,accessibilite_weight,pop_gp
105136,61290000,60730000,14,17,60730000,060730000,06073,C,15.983300,14.973627,42.952587,55.938515,20.985765,96.990200,145.970350,117.074715,107.095375,35.035560,0.130029,673.634104
105137,61290000,60800000,10,18,60800000,060800000,06080,C,4.952830,1.981132,3.962264,3.962264,2.971698,10.896227,18.820755,16.839622,35.660378,4.952830,0.115325,123.157176
105138,61290000,60820000,23,30,60820000,060820000,06082,C,1.019608,1.019608,6.117647,5.098039,4.078432,16.313726,27.529411,14.274509,23.450981,5.098039,0.027324,111.912706
105139,61290000,61020000,13,21,61020000,061020000,06102,C,4.363637,1.454546,3.636364,8.727273,3.636364,10.909091,18.181818,26.909090,24.727272,9.454545,0.080460,128.977663
105140,61290000,61100000,12,19,61100000,061100000,06110,C,3.000000,3.000000,3.000000,7.000000,5.000000,12.000000,25.000000,36.000000,17.000000,8.000000,0.102284,129.742127
105141,61290000,61110000,10,17,61110000,061110000,06111,C,0.000000,0.000000,1.949580,3.899160,2.924370,15.596639,23.394958,27.294117,26.319328,14.621849,0.130029,144.181532
105142,61290000,61290000,0,1,61290000,061290000,06129,C,12.756727,9.751091,21.423054,20.561964,12.997735,51.546980,55.843346,32.213420,72.667990,17.237692,0.886920,329.992782
105143,61290000,61460000,18,23,61460000,061460000,06146,C,5.000000,3.000000,11.000000,16.000000,7.000000,23.000000,35.000000,28.000000,13.000000,8.000000,0.063292,144.694340
105144,61290000,61530000,14,21,61530000,061530000,06153,C,18.000000,23.000000,44.000000,67.000000,41.000000,120.000000,165.000000,152.000000,156.000000,48.000000,0.080460,880.919896


In [19]:
apl = apl.drop(["iris", "iris_string", "code_commune"], axis=1)
apl = apl.drop(cols, axis=1)
apl.head(5)

,iris1,iris2,km,time,type_iris,accessibilite_weight,pop_gp
0,40080000,60530000,17,22,C,0.071361,166.271356
1,40080000,60990000,21,24,C,0.056135,1911.697288
2,40390000,61540000,24,30,C,0.027324,500.115348
3,40420000,60530000,11,19,C,0.102284,166.271356
4,40420000,60710000,22,30,C,0.027324,681.401551


In [20]:
apl["iris"] = apl["iris1"]
apl.sort_values(by='iris2')
apl = apl.merge(df, on="iris", how="left", suffixes=('', ''))
apl

,iris1,iris2,km,time,type_iris,accessibilite_weight,pop_gp,iris,id,nom,prenom,dept_id,adresse_norm_id,lon,lat,key,nb_cabinet,weight,nb
0,40080000,60530000,17,22,C,0.071361,166.271356,40080000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,40080000,60990000,21,24,C,0.056135,1911.697288,40080000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,40390000,61540000,24,30,C,0.027324,500.115348,40390000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,40420000,60530000,11,19,C,0.102284,166.271356,40420000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,40420000,60710000,22,30,C,0.027324,681.401551,40420000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117764,831380000,60850103,34,29,A,0.030807,2870.777975,831380000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
117765,831380000,60950101,18,19,A,0.102284,3441.767196,831380000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
117766,831380000,60950102,19,22,A,0.071361,2657.140709,831380000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
117767,831380000,60950103,18,20,A,0.090718,2674.319908,831380000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
apl["nb"] = apl["nb"].fillna(0)
apl = apl.sort_values(by=['iris1', "iris2"])
apl["nb"].unique()

array([ 0.        ,  2.        ,  1.        ,  8.66666667,  4.        ,
        1.5       , 13.        ,  3.        ,  0.5       , 23.        ,
       11.5       ,  5.        ,  6.        , 10.        ,  7.        ,
        4.5       , 11.        , 14.        ,  7.5       , 17.        ,
        3.5       ,  7.16666667,  2.5       ,  6.5       , 10.5       ,
        3.16666667,  1.25      ,  8.        , 13.5       , 12.5       ,
       25.66666667,  8.5       ,  9.        , 11.66666667,  7.25      ,
        5.33333333,  5.58333333,  9.5       ])

In [22]:
apl = apl.sort_values(by='iris1')
apl["wpop"] = apl["accessibilite_weight"] * apl["pop_gp"]
apl["swpop"] = apl.groupby("iris1")["wpop"].transform("sum")
apl["R"] = apl["nb"] / (apl["swpop"] / 100000)
apl.head(5)
# apl["R"].unique()

,iris1,iris2,km,time,type_iris,accessibilite_weight,pop_gp,iris,id,nom,prenom,dept_id,adresse_norm_id,lon,lat,key,nb_cabinet,weight,nb,wpop,swpop,R
0,40080000,60530000,17,22,C,0.071361,166.271356,40080000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,11.865335,119.178009,0.0
1,40080000,60990000,21,24,C,0.056135,1911.697288,40080000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,107.312674,119.178009,0.0
2,40390000,61540000,24,30,C,0.027324,500.115348,40390000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,13.665013,13.665013,0.0
3,40420000,60530000,11,19,C,0.102284,166.271356,40420000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,17.006934,144.602239,0.0
4,40420000,60710000,22,30,C,0.027324,681.401551,40420000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,18.618427,144.602239,0.0


In [23]:
rgp = apl[apl["iris1"] == apl["iris2"]]
rgp = rgp[["iris1", "iris2", "type_iris", "pop_gp", "nb", "R", "swpop", "wpop", "pop_gp"]].copy()
rgp

,iris1,iris2,type_iris,pop_gp,nb,R,swpop,wpop,pop_gp
76,60010000,60010000,C,99.689889,0.0,0.000000,284.966600,88.417000,99.689889
87,60020000,60020000,C,61.061511,0.0,0.000000,248.156915,54.156702,61.061511
98,60030000,60030000,C,626.690248,0.0,0.000000,1152.763026,555.824388,626.690248
112,60040101,60040101,C,1645.886590,0.0,0.000000,188636.780921,1459.770453,1645.886590
477,60040102,60040102,A,2459.845445,2.0,1.327523,150656.508586,2181.687196,2459.845445
...,...,...,...,...,...,...,...,...,...
115797,61610105,61610105,A,3676.152857,5.0,2.466411,202723.702472,3260.455098,3676.152857
116147,61610106,61610106,A,3787.415420,0.0,0.000000,146843.090527,3359.136138,3787.415420
116501,61610107,61610107,A,2646.705909,0.0,0.000000,160873.268995,2347.417561,2646.705909
116506,61620000,61620000,C,786.184842,0.5,26.142175,1912.618206,697.283403,786.184842


In [24]:
apl = apl.merge(rgp, on="iris2", suffixes=("", "_dest"))
apl.head(5)

,iris1,iris2,km,time,type_iris,accessibilite_weight,pop_gp,iris,id,nom,prenom,dept_id,adresse_norm_id,lon,lat,key,nb_cabinet,weight,nb,wpop,swpop,R,iris1_dest,type_iris_dest,pop_gp_dest,nb_dest,R_dest,swpop_dest,wpop_dest,pop_gp_dest
0,40080000,60530000,17,22,C,0.071361,166.271356,40080000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,11.865335,119.178009,0.0,60530000,C,166.271356,0.0,0.0,371.923118,147.469464,166.271356
1,40420000,60530000,11,19,C,0.102284,166.271356,40420000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,17.006934,144.602239,0.0,60530000,C,166.271356,0.0,0.0,371.923118,147.469464,166.271356
2,40760000,60530000,15,19,C,0.102284,166.271356,40760000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,17.006934,767.910183,0.0,60530000,C,166.271356,0.0,0.0,371.923118,147.469464,166.271356
3,40900000,60530000,22,28,C,0.034735,166.271356,40900000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,5.775479,58.010165,0.0,60530000,C,166.271356,0.0,0.0,371.923118,147.469464,166.271356
4,41740000,60530000,14,20,C,0.090718,166.271356,41740000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,15.083797,151.504943,0.0,60530000,C,166.271356,0.0,0.0,371.923118,147.469464,166.271356


In [25]:
apl = apl.sort_values(by=['iris1', "iris2"])
apl["ap"] = apl["accessibilite_weight"] * apl["R_dest"]
apl["apl"] = apl.groupby("iris1")["ap"].transform("sum")
apl.head(5)

,iris1,iris2,km,time,type_iris,accessibilite_weight,pop_gp,iris,id,nom,prenom,dept_id,adresse_norm_id,lon,lat,key,nb_cabinet,weight,nb,wpop,swpop,R,iris1_dest,type_iris_dest,pop_gp_dest,nb_dest,R_dest,swpop_dest,wpop_dest,pop_gp_dest,ap,apl
0,40080000,60530000,17,22,C,0.071361,166.271356,40080000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,11.865335,119.178009,0.0,60530000,C,166.271356,0.0,0.000000,371.923118,147.469464,166.271356,0.000000,2.736847
12,40080000,60990000,21,24,C,0.056135,1911.697288,40080000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,107.312674,119.178009,0.0,60990000,C,1911.697288,1.0,48.754947,2051.073916,1695.523394,1911.697288,2.736847,2.736847
33,40390000,61540000,24,30,C,0.027324,500.115348,40390000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,13.665013,13.665013,0.0,61540000,C,500.115348,1.0,113.902494,877.943901,443.562523,500.115348,3.112240,3.112240
1,40420000,60530000,11,19,C,0.102284,166.271356,40420000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,17.006934,144.602239,0.0,60530000,C,166.271356,0.0,0.000000,371.923118,147.469464,166.271356,0.000000,5.221543
51,40420000,60710000,22,30,C,0.027324,681.401551,40420000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,18.618427,144.602239,0.0,60710000,C,681.401551,1.0,90.935476,1099.680828,604.348961,681.401551,2.484696,5.221543


In [26]:
apl = apl[apl["iris1"] == apl["iris2"]]
apl

,iris1,iris2,km,time,type_iris,accessibilite_weight,pop_gp,iris,id,nom,prenom,dept_id,adresse_norm_id,lon,lat,key,nb_cabinet,weight,nb,wpop,swpop,R,iris1_dest,type_iris_dest,pop_gp_dest,nb_dest,R_dest,swpop_dest,wpop_dest,pop_gp_dest,ap,apl
385,60010000,60010000,0,1,C,0.88692,99.689889,60010000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,88.417000,284.966600,0.000000,60010000,C,99.689889,0.0,0.000000,284.966600,88.417000,99.689889,0.000000,35.057748
157,60020000,60020000,0,1,C,0.88692,61.061511,60020000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,54.156702,248.156915,0.000000,60020000,C,61.061511,0.0,0.000000,248.156915,54.156702,61.061511,0.000000,0.000000
242,60030000,60030000,0,1,C,0.88692,626.690248,60030000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,555.824388,1152.763026,0.000000,60030000,C,626.690248,0.0,0.000000,1152.763026,555.824388,626.690248,0.000000,18.827956
74748,60040101,60040101,0,1,C,0.88692,1645.886590,60040101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1459.770453,188636.780921,0.000000,60040101,C,1645.886590,0.0,0.000000,188636.780921,1459.770453,1645.886590,0.000000,141.051615
74398,60040102,60040102,0,1,A,0.88692,2459.845445,60040102,16425.0,DAEMERS,YVES,6.0,92989.0,7.072160,43.596934,16425_43.596934_7.07216,1.0,1.0,2.0,2181.687196,150656.508586,1.327523,60040102,A,2459.845445,2.0,1.327523,150656.508586,2181.687196,2459.845445,1.177407,112.646918
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39136,61610105,61610105,0,1,A,0.88692,3676.152857,61610105,99028.0,BENASSAYAG,JEAN JACQUES,6.0,68080.0,7.140329,43.639824,99028_43.639824_7.140329,1.0,1.0,5.0,3260.455098,202723.702472,2.466411,61610105,A,3676.152857,5.0,2.466411,202723.702472,3260.455098,3676.152857,2.187510,147.927969
38772,61610106,61610106,0,1,A,0.88692,3787.415420,61610106,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,3359.136138,146843.090527,0.000000,61610106,A,3787.415420,0.0,0.000000,146843.090527,3359.136138,3787.415420,0.000000,107.182161
38423,61610107,61610107,0,1,A,0.88692,2646.705909,61610107,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,2347.417561,160873.268995,0.000000,61610107,A,2646.705909,0.0,0.000000,160873.268995,2347.417561,2646.705909,0.000000,118.240934
116175,61620000,61620000,0,1,C,0.88692,786.184842,61620000,16814.0,DI VINCENZO,DOMINIQUE,6.0,273951.0,7.612673,44.063665,16814_44.063665_7.612673,2.0,0.5,0.5,697.283403,1912.618206,26.142175,61620000,C,786.184842,0.5,26.142175,1912.618206,697.283403,786.184842,23.186029,61.291546


In [27]:
apl["apl"].describe() #mean=95 vs 98, median=101 vs 110

count    500.000000
mean      95.659051
std       50.279046
min        0.000000
25%       62.585273
50%      100.986393
75%      134.560344
max      459.556143
Name: apl, dtype: float64

In [28]:
apl[apl["apl"]>400]

,iris1,iris2,km,time,type_iris,accessibilite_weight,pop_gp,iris,id,nom,prenom,dept_id,adresse_norm_id,lon,lat,key,nb_cabinet,weight,nb,wpop,swpop,R,iris1_dest,type_iris_dest,pop_gp_dest,nb_dest,R_dest,swpop_dest,wpop_dest,pop_gp_dest,ap,apl
116464,61290000,61290000,0,1,C,0.88692,329.992782,61290000,16295.0,PARODI CARIZZONI,CATHERINE,6.0,264607.0,7.103691,44.08307,16295_44.08307_7.103691,1.0,1.0,2.5,292.677342,519.962334,480.804057,61290000,C,329.992782,2.5,480.804057,519.962334,292.677342,329.992782,426.434944,459.556143


In [29]:
apl = apl[["iris1", "type_iris", "nb", "apl", "ap", "accessibilite_weight", "wpop", "swpop", "R", "pop_gp"]]
apl = apl.rename(columns={'iris1': 'iris'})
apl.head(5)

,iris,type_iris,nb,apl,ap,accessibilite_weight,wpop,swpop,R,pop_gp
385,60010000,C,0.0,35.057748,0.000000,0.88692,88.417000,284.966600,0.000000,99.689889
157,60020000,C,0.0,0.000000,0.000000,0.88692,54.156702,248.156915,0.000000,61.061511
242,60030000,C,0.0,18.827956,0.000000,0.88692,555.824388,1152.763026,0.000000,626.690248
74748,60040101,C,0.0,141.051615,0.000000,0.88692,1459.770453,188636.780921,0.000000,1645.886590
74398,60040102,A,2.0,112.646918,1.177407,0.88692,2181.687196,150656.508586,1.327523,2459.845445


In [30]:
def get_iriss():
    sql = f"""
select i.id "iris", i.code "iris_string", i.nom "iris_label", c.dept_id "dept", c.code "code_commune", c.nom "commune_label", 20{year} "year", {specialite} "specialite" from iris.iris i
join iris.commune c on c.id=i.commune_id
where i.code like '{dept:02d}%'"""
    print(sql)
    return pd.read_sql(text(sql), config.connection_string)

iriss = get_iriss()
iriss


select i.id "iris", i.code "iris_string", i.nom "iris_label", c.dept_id "dept", c.code "code_commune", c.nom "commune_label", 2021 "year", 10 "specialite" from iris.iris i
join iris.commune c on c.id=i.commune_id
where i.code like '06%'


,iris,iris_string,iris_label,dept,code_commune,commune_label,year,specialite
0,60120105,060120105,Centre,6,06012,Beausoleil,2021,10
1,60500000,060500000,Coursegoules,6,06050,Coursegoules,2021,10
2,60720000,060720000,Ilonse,6,06072,Ilonse,2021,10
3,61050101,061050101,Nord,6,06105,Roquefort-les-Pins,2021,10
4,60150000,060150000,Berre-les-Alpes,6,06015,Berre-les-Alpes,2021,10
...,...,...,...,...,...,...,...,...
495,60883403,060883403,Madeleine Supérieure,6,06088,Nice,2021,10
496,60883501,060883501,Crémat-Bellet,6,06088,Nice,2021,10
497,60883601,060883601,Saint-Pierre de Féric,6,06088,Nice,2021,10
498,60883602,060883602,Costière,6,06088,Nice,2021,10


In [31]:
apl = apl.merge(iriss, on="iris", how="left", suffixes=("", "_dest"))
apl = apl[["year", "specialite", "iris", "iris_string", "type_iris", "nb", "apl", "ap", "accessibilite_weight", "R", "wpop", "swpop", "pop_gp", "iris_label", "dept", "code_commune", "commune_label"]]
apl

,year,specialite,iris,iris_string,type_iris,nb,apl,ap,accessibilite_weight,R,wpop,swpop,pop_gp,iris_label,dept,code_commune,commune_label
0,2021,10,60010000,060010000,C,0.0,35.057748,0.000000,0.88692,0.000000,88.417000,284.966600,99.689889,Aiglun,6,06001,Aiglun
1,2021,10,60020000,060020000,C,0.0,0.000000,0.000000,0.88692,0.000000,54.156702,248.156915,61.061511,Amirat,6,06002,Amirat
2,2021,10,60030000,060030000,C,0.0,18.827956,0.000000,0.88692,0.000000,555.824388,1152.763026,626.690248,Andon,6,06003,Andon
3,2021,10,60040101,060040101,C,0.0,141.051615,0.000000,0.88692,0.000000,1459.770453,188636.780921,1645.886590,Les Trois Moulins,6,06004,Antibes
4,2021,10,60040102,060040102,A,2.0,112.646918,1.177407,0.88692,1.327523,2181.687196,150656.508586,2459.845445,Les Semboules,6,06004,Antibes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,2021,10,61610105,061610105,A,5.0,147.927969,2.187510,0.88692,2.466411,3260.455098,202723.702472,3676.152857,Marina,6,06161,Villeneuve-Loubet
496,2021,10,61610106,061610106,A,0.0,107.182161,0.000000,0.88692,0.000000,3359.136138,146843.090527,3787.415420,Vaugrenier,6,06161,Villeneuve-Loubet
497,2021,10,61610107,061610107,A,0.0,118.240934,0.000000,0.88692,0.000000,2347.417561,160873.268995,2646.705909,Plan,6,06161,Villeneuve-Loubet
498,2021,10,61620000,061620000,C,0.5,61.291546,23.186029,0.88692,26.142175,697.283403,1912.618206,786.184842,La Brigue,6,06162,La Brigue


In [32]:
# Anomalie = 1 seul medecin avec 1 seul cabinet pour 300 habitants contre 1 seul medecin et 0.33 cabinets
# La commune de sauveur en tinee a un APL > 400 pour les généralistes car l'iris a seulement 300 habitants jeunes et ils sont passés de 1 médecin avec 3 cabinets à 1 médecin a un seul cabinet en 2022
apl[apl["apl"] > 400]


,year,specialite,iris,iris_string,type_iris,nb,apl,ap,accessibilite_weight,R,wpop,swpop,pop_gp,iris_label,dept,code_commune,commune_label
436,2021,10,61290000,061290000,C,2.5,459.556143,426.434944,0.88692,480.804057,292.677342,519.962334,329.992782,Saint-Sauveur-sur-Tinée,6,06129,Saint-Sauveur-sur-Tinée
